In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
from torch import Tensor
import dlc_practical_prologue as prologue

In [ ]:
import dlc_practical_prologue as prologue

train_input, train_target, test_input, test_target = prologue.load_data()

print('train_input', train_input.size(), 'train_target', train_target.size())
print('test_input', test_input.size(), 'test_target', test_target.size())

* Using MNIST
Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to ./data/mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/mnist/MNIST/raw

Processing...


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!
** Reduce the data-set (use --full for the full thing)
** Use 1000 train and 1000 test samples
train_input torch.Size([1000, 784]) train_target torch.Size([1000])
test_input torch.Size([1000, 784]) test_target torch.Size([1000])


In [ ]:
 # Question 1

In [ ]:
"""
train_input is a 2d float tensor of dimension n x d containing the training vectors,
train_target is a 1d long tensor of dimension n containing the training labels,
x is 1d float tensor of dimension d containing the test vector,
and the returned value is the class of the train sample closest to x for the L2 norm.
"""
def nearest_classification(train_input, train_target, x):
  term = (train_input - x).pow(2)
  term = term.sum(1)
  distance = term.pow(0.5)
  
  min_val, min_index = torch.min(distance, 0, keepdim=False, out=None)
  return train_target[min_index]

In [ ]:
# Test
predicted_label = nearest_classification(train_input,train_target,test_input[571])
print(test_target[44])
print(predicted_label)

tensor(3)
tensor(4)


In [ ]:
# Question 2

In [ ]:
"""
train_input is a 2d float tensor of dimension n x d containing the train vectors,
train_target is a 1d long tensor of dimension n containing the train labels,
test_input is a 2d float tensor of dimension m x d containing the test vectors,
test_target is a 1d long tensor of dimension m containing the test labels,
mean is either None or a 1d float tensor of dimension d,
proj is either None or a 2d float tensor of dimension c x d,
"""
def compute_nb_errors(train_input, train_target, test_input, test_target, mean = None, proj = None):
  if(mean != None):
    train_input -= mean # train_input.sub(mean)
    test_input -= mean # test_input.sub(mean)
  if(proj != None):
    # all works but HOW?

    #train_input = train_input.mm(proj.t())
    #test_input = test_input.mm(proj.t())
    
    #train_input = train_input @ proj.T 
    #test_input  = test_input  @ proj.T

    train_input = train_input.mm(torch.transpose(proj, 0, 1))
    test_input = test_input.mm(torch.transpose(proj, 0, 1))

    #train_input = torch.mm(train_input, proj.trn())
    #test_input = torch.mm(test_input, proj.trn())

    #train_input = train_input.mm(proj)
    #test_input = test_input.mm(proj)

  nb_errors = 0
  for i in range(test_input.size(0)):
    if(test_target[i] != nearest_classification(train_input,train_target,test_input[i])):
      nb_errors += 1
  return nb_errors

result = compute_nb_errors(train_input, train_target, test_input[:100, :], test_target[:100])

print(result)

17


In [ ]:
# Question 3
# https://github.com/charlielu05/EE-559-Pracs/blob/master/prac_2/dlc_practical_2_solution.py

In [ ]:
def PCA(x):
    mean = x.mean(0)
    b = x - mean
    Sigma = b.t().mm(b)
    eigen_values, eigen_vectors = Sigma.eig(True)
    right_order = eigen_values[:,0].abs().sort(0, True)[1]
    eigen_vectors = eigen_vectors.t()[right_order]
    return mean, eigen_vectors


In [ ]:
for c in [ False, True ]:

    train_input, train_target, test_input, test_target = prologue.load_data(cifar=c)

    nb_errors = compute_nb_errors(train_input, train_target, test_input, test_target)
    print('Baseline nb_errors {:d} error {:.02f}%'.format(nb_errors, 100 * nb_errors / test_input.size(0)))

    ##

    basis = train_input.new(100, train_input.size(1)).normal_()

    nb_errors = compute_nb_errors(train_input, train_target, test_input, test_target, None, basis)
    print('Random {:d}d nb_errors {:d} error {:.02f}%'.format(basis.size(0), nb_errors, 100 * nb_errors / test_input.size(0)))

    ##

    mean, basis = PCA(train_input)

    for d in [ 100, 50, 10, 3 ]:
        basis = basis.narrow(0, 0, d)
        nb_errors = compute_nb_errors(train_input, train_target, test_input, test_target, mean, basis)
        print('PCA {:d}d nb_errors {:d} error {:.02f}%'.format(d, nb_errors, 100 * nb_errors / test_input.size(0)))


* Using MNIST
** Reduce the data-set (use --full for the full thing)
** Use 1000 train and 1000 test samples
Baseline nb_errors 172 error 17.20%
Random 100d nb_errors 212 error 21.20%
PCA 100d nb_errors 164 error 16.40%
PCA 50d nb_errors 155 error 15.50%
PCA 10d nb_errors 214 error 21.40%
PCA 3d nb_errors 597 error 59.70%
* Using CIFAR



Extracting ./data/cifar10/cifar-10-python.tar.gz to ./data/cifar10/
Files already downloaded and verified
** Reduce the data-set (use --full for the full thing)
** Use 1000 train and 1000 test samples
Baseline nb_errors 746 error 74.60%
Random 100d nb_errors 762 error 76.20%
PCA 100d nb_errors 745 error 74.50%
PCA 50d nb_errors 737 error 73.70%
PCA 10d nb_errors 763 error 76.30%
PCA 3d nb_errors 839 error 83.90%
